In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
BASE = "/content/drive/MyDrive/helpdesk_enterprise_project"
os.makedirs(BASE, exist_ok=True)
print("Project base:", BASE)

Mounted at /content/drive
Project base: /content/drive/MyDrive/helpdesk_enterprise_project


In [2]:
import os
folders = ["src","data","notebooks","docs","diagrams"]
for f in folders:
    os.makedirs(os.path.join(BASE,f), exist_ok=True)
!ls -la "{BASE}"
print("Folders created.")

total 21
drwx------ 2 root root 4096 Nov 25 18:39 data
drwx------ 2 root root 4096 Nov 25 18:39 diagrams
drwx------ 2 root root 4096 Nov 25 18:39 docs
drwx------ 2 root root 4096 Nov 25 18:39 notebooks
-rw------- 1 root root  404 Nov 30 17:16 README.md
-rw------- 1 root root  104 Nov 30 17:16 requirements.txt
drwx------ 2 root root 4096 Nov 25 18:39 src
Folders created.


In [3]:
# Write schema.sql
schema = r"""PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS departments (
    department_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE
);

CREATE TABLE IF NOT EXISTS agents (
    agent_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    department_id INTEGER,
    role TEXT,
    FOREIGN KEY(department_id) REFERENCES departments(department_id)
);

CREATE TABLE IF NOT EXISTS priorities (
    priority_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    sla_target_minutes INTEGER NOT NULL
);

CREATE TABLE IF NOT EXISTS tickets (
    ticket_id INTEGER PRIMARY KEY AUTOINCREMENT,
    created_at TEXT NOT NULL,
    department_id INTEGER,
    priority_id INTEGER,
    status TEXT NOT NULL,
    reporter TEXT,
    assignee_id INTEGER,
    resolved_at TEXT,
    sla_breached INTEGER DEFAULT 0,
    resolution_minutes INTEGER,
    subject TEXT,
    description TEXT,
    FOREIGN KEY(department_id) REFERENCES departments(department_id),
    FOREIGN KEY(priority_id) REFERENCES priorities(priority_id),
    FOREIGN KEY(assignee_id) REFERENCES agents(agent_id)
);

CREATE INDEX IF NOT EXISTS idx_tickets_created_at ON tickets(created_at);
CREATE INDEX IF NOT EXISTS idx_tickets_priority ON tickets(priority_id);
CREATE INDEX IF NOT EXISTS idx_tickets_status ON tickets(status);
"""
open(os.path.join(BASE,"src","schema.sql"),"w").write(schema)

# Write generate_data.py
gen = r'''#!/usr/bin/env python3
"""
Realistic helpdesk data generator.
Generates departments, agents, priorities and ~600 tickets as a Poisson arrival process.
"""
import sqlite3, random, datetime, os
from faker import Faker
import numpy as np

fake = Faker()
DB = "data/tickets.db"

def ensure_db():
    os.makedirs("data", exist_ok=True)
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    with open("src/schema.sql", "r") as f:
        cur.executescript(f.read())
    conn.commit()
    conn.close()

def seed_reference_data():
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    depts = ['IT Support','Facilities','HR','Finance','Security']
    cur.execute("DELETE FROM departments;")
    for d in depts:
        cur.execute("INSERT OR IGNORE INTO departments(name) VALUES (?)",(d,))
    priorities = [('Low', 72*60), ('Medium', 48*60), ('High', 8*60), ('Urgent', 60)]
    cur.execute("DELETE FROM priorities;")
    for name, sla in priorities:
        cur.execute("INSERT OR IGNORE INTO priorities(name, sla_target_minutes) VALUES (?,?)",(name,sla))
    cur.execute("DELETE FROM agents;")
    conn.commit()
    cur.execute("SELECT department_id FROM departments;")
    dept_ids = [r[0] for r in cur.fetchall()]
    for did in dept_ids:
        for i in range(3):
            cur.execute("INSERT INTO agents(name, department_id, role) VALUES (?,?,?)",
                        (fake.name(), did, random.choice(['analyst','engineer','lead'])))
    conn.commit()
    conn.close()

def simulate_arrivals(days=90, lam_per_day=30):
    """Poisson process with weekday/weekend modulation over N days."""
    arrivals = []
    start = datetime.datetime.now() - datetime.timedelta(days=days)
    for day in range(days):
        date = start + datetime.timedelta(days=day)
        weekday = date.weekday()
        lam = lam_per_day * (1.2 if weekday < 5 else 0.6)
        n = np.random.poisson(lam)
        for i in range(n):
            minute = random.randint(0, 24*60-1)
            ts = date + datetime.timedelta(minutes=minute)
            arrivals.append(ts)
    arrivals.sort()
    return arrivals

def generate_tickets(n=None):
    conn = sqlite3.connect(DB)
    cur = conn.cursor()
    cur.execute("DELETE FROM tickets;")
    cur.execute("SELECT department_id FROM departments;")
    depts = [r[0] for r in cur.fetchall()]
    cur.execute("SELECT priority_id, sla_target_minutes FROM priorities;")
    prios = cur.fetchall()
    cur.execute("SELECT agent_id FROM agents;")
    agents = [r[0] for r in cur.fetchall()]
    arrivals = simulate_arrivals(days=90, lam_per_day=30)
    if n is not None:
        arrivals = arrivals[:n]
    for created in arrivals:
        dept = random.choice(depts)
        priority_id, sla = random.choices(prios, weights=[0.5,0.3,0.15,0.05])[0]
        status = random.choices(['new','triage','assigned','resolved','closed','reopened'],
                                weights=[0.05,0.1,0.45,0.25,0.12,0.03])[0]
        assignee = random.choice(agents) if random.random() < 0.7 else None
        resolved = None
        resolution_minutes = None
        sla_breached = 0
        if status in ('resolved','closed'):
            lam = max(60, int(sla/2))
            resolution_minutes = int(np.random.exponential(lam) + random.randint(30,120))
            resolved = created + datetime.timedelta(minutes=resolution_minutes)
            sla_breached = 1 if resolution_minutes > sla else 0
        subject = fake.sentence(nb_words=6)
        desc = fake.paragraph(nb_sentences=3)
        cur.execute("""INSERT INTO tickets (created_at, department_id, priority_id, status, reporter, assignee_id, resolved_at, sla_breached, resolution_minutes, subject, description)
                       VALUES (?,?,?,?,?,?,?,?,?,?,?)""",
                    (created.isoformat(), dept, priority_id, status, fake.email(), assignee, resolved.isoformat() if resolved else None, sla_breached, resolution_minutes, subject, desc))
    conn.commit()
    conn.close()

if __name__ == '__main__':
    ensure_db()
    seed_reference_data()
    generate_tickets()
    print("Generated realistic helpdesk data at", DB)
'''
open(os.path.join(BASE,"src","generate_data.py"),"w").write(gen)
os.chmod(os.path.join(BASE,"src","generate_data.py"), 0o755)

# Write Streamlit app
app = r'''import streamlit as st
import pandas as pd, sqlite3
import plotly.express as px

DB = "data/tickets.db"

@st.cache_data(ttl=600)
def load_data():
    conn = sqlite3.connect(DB)
    df = pd.read_sql_query(
        "SELECT t.*, p.name as priority, p.sla_target_minutes, d.name as department, a.name as assignee_name "
        "FROM tickets t "
        "LEFT JOIN priorities p ON t.priority_id = p.priority_id "
        "LEFT JOIN departments d ON t.department_id = d.department_id "
        "LEFT JOIN agents a ON t.assignee_id = a.agent_id",
        conn, parse_dates=['created_at','resolved_at'])
    conn.close()
    return df

st.set_page_config(layout="wide", page_title="Helpdesk Enterprise Dashboard")
st.title("Helpdesk Enterprise Dashboard — KPI & Analytics")
if st.button("Regenerate data (dev)"):
    import subprocess, sys
    subprocess.check_call([sys.executable, "src/generate_data.py"])
    st.experimental_rerun()

df = load_data()
st.sidebar.header("Filters")
dept = st.sidebar.multiselect("Department", sorted(df['department'].dropna().unique().tolist()))
prio = st.sidebar.multiselect("Priority", sorted(df['priority'].dropna().unique().tolist()))
status = st.sidebar.multiselect("Status", sorted(df['status'].dropna().unique().tolist()))

fdf = df.copy()
if dept: fdf = fdf[fdf['department'].isin(dept)]
if prio: fdf = fdf[fdf['priority'].isin(prio)]
if status: fdf = fdf[fdf['status'].isin(status)]

col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Tickets", len(fdf))
col2.metric("Open Tickets", len(fdf[fdf['status'].isin(['new','assigned','triage','reopened'])]))
col3.metric("SLA Breached", int(fdf['sla_breached'].sum()))
col4.metric("Avg Resolution (hrs)", round(fdf['resolution_minutes'].dropna().mean()/60 if fdf['resolution_minutes'].count()>0 else 0,1))

with st.expander("Priority Distribution"):
    fig = px.histogram(fdf, x='priority', category_orders={'priority':['Low','Medium','High','Urgent']}, title="Priority Mix")
    st.plotly_chart(fig, use_container_width=True)

with st.expander("Resolution Time (minutes) distribution"):
    fig = px.histogram(fdf[fdf['resolution_minutes'].notnull()], x='resolution_minutes', nbins=50, title="Resolution Minutes")
    st.plotly_chart(fig, use_container_width=True)

with st.expander("Tickets over time"):
    daily = fdf.assign(day=lambda d: d['created_at'].dt.date).groupby('day').size().reset_index(name='count')
    fig = px.line(daily, x='day', y='count', title="Daily Ticket Volume")
    st.plotly_chart(fig, use_container_width=True)

st.write("Sample rows:")
st.dataframe(fdf.head(100))

csv = fdf.to_csv(index=False)
st.download_button("Download CSV", csv, file_name="tickets_export.csv", mime='text/csv')
'''
open(os.path.join(BASE,"src","app.py"),"w").write(app)

# write requirements, README, governance memo
open(os.path.join(BASE,"requirements.txt"),"w").write("pandas\nnumpy\nfaker\nsqlalchemy\nstreamlit\nplotly\nmatplotlib\nseaborn\npyngrok\n")
open(os.path.join(BASE,"README.md"),"w").write("# Helpdesk Enterprise Project\n\nGenerated by master Colab script.\n")
open(os.path.join(BASE,"docs","governance_memo.md"),"w").write("# Governance Memo\n\nSee project documentation.\n")

# write ERD and process map drawio XML files
erd_xml = r'''<?xml version="1.0" encoding="UTF-8"?>
<mxfile host="app.diagrams.net">
  <diagram name="Helpdesk ER Diagram">
    <mxGraphModel dx="1196" dy="772" grid="1" gridSize="10" guides="1" tooltips="1" connect="1" arrows="1" fold="1" page="1" pageScale="1" pageWidth="827" pageHeight="1169">
      <root>
        <mxCell id="0"/>
        <mxCell id="1" parent="0"/>
        <mxCell id="departments" value="&lt;b&gt;departments&lt;/b&gt;&lt;br/&gt;- department_id &lt;i&gt;(PK)&lt;/i&gt;&lt;br/&gt;- name &lt;i&gt;(UNIQUE, NOT NULL)&lt;/i&gt;" style="html=1;rounded=1;whiteSpace=wrap;align=left;verticalAlign=top;fillColor=#dae8fc;strokeColor=#6c8ebf;fontSize=12;" vertex="1" parent="1">
          <mxGeometry x="40" y="80" width="240" height="80" as="geometry"/>
        </mxCell>
        <mxCell id="agents" value="&lt;b&gt;agents&lt;/b&gt;&lt;br/&gt;- agent_id &lt;i&gt;(PK)&lt;/i&gt;&lt;br/&gt;- name &lt;i&gt;(NOT NULL)&lt;/i&gt;&lt;br/&gt;- department_id &lt;i&gt;(FK)&lt;/i&gt;&lt;br/&gt;- role" style="html=1;rounded=1;whiteSpace=wrap;align=left;verticalAlign=top;fillColor=#fff2cc;strokeColor=#d6b656;fontSize=12;" vertex="1" parent="1">
          <mxGeometry x="320" y="80" width="260" height="110" as="geometry"/>
        </mxCell>
        <mxCell id="priorities" value="&lt;b&gt;priorities&lt;/b&gt;&lt;br/&gt;- priority_id &lt;i&gt;(PK)&lt;/i&gt;&lt;br/&gt;- name &lt;i&gt;(UNIQUE, NOT NULL)&lt;/i&gt;&lt;br/&gt;- sla_target_minutes &lt;i&gt;(NOT NULL)&lt;/i&gt;" style="html=1;rounded=1;whiteSpace=wrap;align=left;verticalAlign=top;fillColor=#d5e8d4;strokeColor=#82b366;fontSize=12;" vertex="1" parent="1">
          <mxGeometry x="40" y="220" width="240" height="100" as="geometry"/>
        </mxCell>
        <mxCell id="tickets" value="&lt;b&gt;tickets&lt;/b&gt;&lt;br/&gt;- ticket_id &lt;i&gt;(PK)&lt;/i&gt;&lt;br/&gt;- created_at &lt;i&gt;(NOT NULL)&lt;/i&gt;&lt;br/&gt;- department_id &lt;i&gt;(FK)&lt;/i&gt;&lt;br/&gt;- priority_id &lt;i&gt;(FK)&lt;/i&gt;&lt;br/&gt;- status &lt;i&gt;(NOT NULL)&lt;/i&gt;&lt;br/&gt;- reporter&lt;br/&gt;- assignee_id &lt;i&gt;(FK)&lt;/i&gt;&lt;br/&gt;- resolved_at&lt;br/&gt;- sla_breached&lt;br/&gt;- resolution_minutes&lt;br/&gt;- subject&lt;br/&gt;- description" style="html=1;rounded=1;whiteSpace=wrap;align=left;verticalAlign=top;fillColor=#f8cecc;strokeColor=#b85450;fontSize=12;" vertex="1" parent="1">
          <mxGeometry x="320" y="220" width="340" height="260" as="geometry"/>
        </mxCell>
        <mxCell id="rel_agents_depts" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;endArrow=block;" edge="1" parent="1" source="agents" target="departments">
          <mxGeometry relative="1" as="geometry"/>
        </mxCell>
        <mxCell id="rel_tickets_depts" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;endArrow=block;" edge="1" parent="1" source="tickets" target="departments">
          <mxGeometry relative="1" as="geometry"/>
        </mxCell>
        <mxCell id="rel_tickets_prios" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;endArrow=block;" edge="1" parent="1" source="tickets" target="priorities">
          <mxGeometry relative="1" as="geometry"/>
        </mxCell>
        <mxCell id="rel_tickets_agents" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;endArrow=block;" edge="1" parent="1" source="tickets" target="agents">
          <mxGeometry relative="1" as="geometry"/>
        </mxCell>
      </root>
    </mxGraphModel>
  </diagram>
</mxfile>
'''
open(os.path.join(BASE,"diagrams","erd.drawio"),"w").write(erd_xml)

process_xml = r'''<?xml version="1.0" encoding="UTF-8"?>
<mxfile host="app.diagrams.net">
  <diagram name="Helpdesk Process Map">
    <mxGraphModel dx="1136" dy="722" grid="1" gridSize="10" guides="1" tooltips="1" connect="1" arrows="1" fold="1" page="1" pageScale="1" pageWidth="827" pageHeight="1169">
      <root>
        <mxCell id="0"/><mxCell id="1" parent="0"/>
        <mxCell id="ticket_submitted" value="Ticket Submitted" style="rounded=1;fillColor=#d5e8d4;strokeColor=#82b366;" vertex="1" parent="1"><mxGeometry x="60" y="80" width="140" height="60" as="geometry"/></mxCell>
        <mxCell id="triage" value="Triage" style="rounded=1;fillColor=#ffe6cc;strokeColor=#d79b00;" vertex="1" parent="1"><mxGeometry x="260" y="80" width="120" height="60" as="geometry"/></mxCell>
        <mxCell id="assign" value="Assign" style="rounded=1;fillColor=#dae8fc;strokeColor=#6c8ebf;" vertex="1" parent="1"><mxGeometry x="420" y="80" width="120" height="60" as="geometry"/></mxCell>
        <mxCell id="work" value="Work" style="rounded=1;fillColor=#fff2cc;strokeColor=#d6b656;" vertex="1" parent="1"><mxGeometry x="580" y="80" width="120" height="60" as="geometry"/></mxCell>
        <mxCell id="resolved" value="Resolved" style="rounded=1;fillColor=#d5e8d4;strokeColor=#82b366;" vertex="1" parent="1"><mxGeometry x="740" y="80" width="120" height="60" as="geometry"/></mxCell>
        <mxCell id="closed" value="Closed" style="rounded=1;fillColor=#f8cecc;strokeColor=#b85450;" vertex="1" parent="1"><mxGeometry x="900" y="80" width="120" height="60" as="geometry"/></mxCell>
        <mxCell id="sla_breach" value="SLA Breach?" style="rhombus;whiteSpace=wrap;html=1;fillColor=#fff2cc;strokeColor=#d79b00;" vertex="1" parent="1"><mxGeometry x="580" y="180" width="120" height="80" as="geometry"/></mxCell>
        <mxCell id="escalate" value="Escalate" style="rounded=1;fillColor=#ffcccc;strokeColor=#b85450;" vertex="1" parent="1"><mxGeometry x="740" y="200" width="120" height="60" as="geometry"/></mxCell>
        <mxCell id="reopened" value="Reopened?" style="rhombus;whiteSpace=wrap;html=1;fillColor=#fff2cc;strokeColor=#d79b00;" vertex="1" parent="1"><mxGeometry x="420" y="180" width="120" height="80" as="geometry"/></mxCell>
        <mxCell id="arrow1" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;" edge="1" parent="1" source="ticket_submitted" target="triage"><mxGeometry relative="1" as="geometry"/></mxCell>
        <mxCell id="arrow2" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;" edge="1" parent="1" source="triage" target="assign"><mxGeometry relative="1" as="geometry"/></mxCell>
        <mxCell id="arrow3" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;" edge="1" parent="1" source="assign" target="work"><mxGeometry relative="1" as="geometry"/></mxCell>
        <mxCell id="arrow4" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;" edge="1" parent="1" source="work" target="resolved"><mxGeometry relative="1" as="geometry"/></mxCell>
        <mxCell id="arrow5" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#000000;" edge="1" parent="1" source="resolved" target="closed"><mxGeometry relative="1" as="geometry"/></mxCell>
        <mxCell id="arrow_sla" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#ff0000;endArrow=classic;" edge="1" parent="1" source="work" target="sla_breach"><mxGeometry relative="1" as="geometry"/></mxCell>
        <mxCell id="arrow_escalate" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#b85450;endArrow=classic;" edge="1" parent="1" source="sla_breach" target="escalate"><mxGeometry relative="1" as="geometry"/></mxCell>
        <mxCell id="arrow_reopen" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;strokeColor=#0000ff;endArrow=classic;" edge="1" parent="1" source="reopened" target="triage"><mxGeometry relative="1" as="geometry"/></mxCell>
      </root>
    </mxGraphModel>
  </diagram>
</mxfile>
'''
open(os.path.join(BASE,"diagrams","process_map.drawio"),"w").write(process_xml)

print("All template files written to project.")


All template files written to project.


In [4]:
# installs used for generation, notebook, and streamlit
!pip install -q pandas numpy faker sqlalchemy matplotlib seaborn plotly streamlit pyngrok nbformat
print("Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.8 MB/s eta 0:00:00
Dependencies installed


In [5]:
%cd "{BASE}"
!python3 src/generate_data.py
# show db info
!ls -lh data/tickets.db

/content/drive/MyDrive/helpdesk_enterprise_project
Generated realistic helpdesk data at data/tickets.db
-rw------- 1 root root 828K Nov 30 17:43 data/tickets.db


In [6]:
import nbformat as nbf
nb = nbf.v4.new_notebook()
cells = []

# Cell 1: install deps
cells.append(nbf.v4.new_code_cell("%pip install -q pandas numpy matplotlib seaborn plotly"))

# Cell 2: load data
cells.append(nbf.v4.new_code_cell('''import sqlite3, pandas as pd
conn = sqlite3.connect('/content/drive/MyDrive/helpdesk_enterprise_project/data/tickets.db')
df = pd.read_sql_query(
    "SELECT t.*, p.name as priority, p.sla_target_minutes, d.name as department FROM tickets t "
    "LEFT JOIN priorities p ON t.priority_id = p.priority_id "
    "LEFT JOIN departments d ON t.department_id = d.department_id",
    conn, parse_dates=['created_at','resolved_at'])
conn.close()
df['created_at'] = pd.to_datetime(df['created_at'])
df['resolved_at'] = pd.to_datetime(df['resolved_at'])
df.head()
'''))

# Cell 3: Weekly volume
cells.append(nbf.v4.new_code_cell('''import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
df.set_index('created_at').resample('7D').size().plot(title='Weekly Ticket Volume')
plt.ylabel('Tickets per week')
plt.show()'''))

# Cell 4: Resolution by priority
cells.append(nbf.v4.new_code_cell('''import seaborn as sns
plt.figure(figsize=(10,4))
sns.boxplot(x='priority', y='resolution_minutes', data=df[df['resolution_minutes'].notnull()], order=['Low','Medium','High','Urgent'])
plt.title('Resolution time by priority (minutes)')
plt.show()'''))

# Cell 5: SLA breach by dept
cells.append(nbf.v4.new_code_cell('''plt.figure(figsize=(10,4))
agg = df.groupby('department').agg({'ticket_id':'count','sla_breached':'sum'}).assign(sla_rate=lambda d:d['sla_breached']/d['ticket_id']*100).sort_values('sla_rate', ascending=False)
agg['sla_rate'].plot(kind='bar')
plt.ylabel('SLA breach rate (%)')
plt.title('SLA breach rate by department')
plt.show()'''))

nb['cells'] = cells
nbf.write(nb, os.path.join(BASE,'notebooks','analysis.ipynb'))
print("analysis.ipynb written to notebooks/ — open it in Colab (File → Open notebook → Google Drive).")

analysis.ipynb written to notebooks/ — open it in Colab (File → Open notebook → Google Drive).


In [7]:
# ---- set
NGROK_TOKEN = ""
# --------------------------------------------------------------

if NGROK_TOKEN == "35ysp5jlTKecdYShfG0Jn8JX885_4eeMcAZXxin7gNthi8CFg":
    print("NGROK_TOKEN is empty. To expose publicly, set NGROK_TOKEN variable and re-run this cell.")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_TOKEN)
    import time, os
    os.chdir("/content/drive/MyDrive/helpdesk_enterprise_project")
    # kill previous streamlit if any
    !pkill -f streamlit || true
    get_ipython().system_raw("streamlit run src/app.py --server.port 8501 --server.headless true &> streamlit.log &")
    public_url = ngrok.connect(8501)
    print("Streamlit public URL:", public_url)
    !sed -n '1,200p' streamlit.log


^C


ERROR:pyngrok.process.ngrok:t=2025-11-30T17:52:31+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-30T17:52:31+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-30T17:52:31+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [8]:
# ---- set your ngrok token string here (replace the value) ----
NGROK_TOKEN = "35ysp5jlTKecdYShfG0Jn8JX885_4eeMcAZXxin7gNthi8CFg"
# --------------------------------------------------------------

if NGROK_TOKEN == "":
    print("NGROK_TOKEN is empty. To expose publicly, set NGROK_TOKEN variable and re-run this cell.")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_TOKEN)
    import time, os
    os.chdir("/content/drive/MyDrive/helpdesk_enterprise_project")
    # kill previous streamlit if any
    !pkill -f streamlit || true
    get_ipython().system_raw("streamlit run src/app.py --server.port 8501 --server.headless true &> streamlit.log &")
    public_url = ngrok.connect(8501)
    print("Streamlit public URL:", public_url)
    !sed -n '1,200p' streamlit.log


^C
Streamlit public URL: NgrokTunnel: "https://macronucleate-sordidly-elli.ngrok-free.dev" -> "http://localhost:8501"


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
